In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Local imports
from src.data_processing.lstm_data_preprocessing import reduce_time_bucket_features, FeaturesConfig
from src.data_processing.loader import load_time_bucket_data

In [3]:
# Define configs
features_config = FeaturesConfig(
        relative_time=True,
        price_change=True,
        trade_size_ratio=True,
        liquidity_ratio=True,
        wallet_trade_size_deviation=True,
        rough_pnl=True,
        average_roi=True,
        win_rate=True,
        average_hold_duration=True
    )

test_size = 0.2

In [4]:
# Generate train - test data

X_scaler = StandardScaler()
y_scaler = StandardScaler()

time_bucket_folder = "time_bucket_1"  # Change based on which time bucket configuration you want to used, their preprocessed in different folders
token_time_buckets, time_bucket_config = load_time_bucket_data(time_bucket_folder)

token_datasets = []
for token_address, data in token_time_buckets.items():
    X = data["X"]
    y = data["y"]
    bucket_times = data["bucket_times"]

    # Only get the features listed in features_config
    X = reduce_time_bucket_features(X, features_config)

    token_datasets.append((X, y, token_address, bucket_times))

# Combine all token data
all_X = np.vstack([data[0] for data in token_datasets])
all_y = np.vstack([data[1].reshape(-1, 1) for data in token_datasets])

# Scale features
num_samples, time_steps, features = all_X.shape
X_reshaped = all_X.reshape(num_samples * time_steps, features)
X_scaled = X_scaler.fit_transform(X_reshaped)
X_scaled = X_scaled.reshape(num_samples, time_steps, features)

# Scale target variable also using StandardScaler to preserve direction
y_scaled = y_scaler.fit_transform(all_y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=test_size, shuffle=False)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Bidirectional, BatchNormalization, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import BayesianOptimization

# Custom loss function
def weighted_mse_large_moves(y_true, y_pred):
    diff = y_true - y_pred
    weight = tf.math.square(y_true)
    return tf.reduce_mean(weight * tf.square(diff))

# Model building function
def build_model(hp):
    model = Sequential()
    
    model.add(Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])))
    
    model.add(Bidirectional(
        LSTM(
            units=hp.Int('lstm_units_1', 32, 128, step=16),
            return_sequences=True,
            kernel_regularizer=regularizers.l2(hp.Choice('l2_reg', [1e-5, 1e-4, 1e-3]))
        )
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', 0.1, 0.5, step=0.1)))
    
    model.add(Bidirectional(
        LSTM(
            units=hp.Int('lstm_units_2', 32, 128, step=16),
            return_sequences=False,
            kernel_regularizer=regularizers.l2(hp.Choice('l2_reg', [1e-5, 1e-4, 1e-3]))
        )
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_2', 0.1, 0.5, step=0.1)))
    
    model.add(Dense(
        hp.Int('dense_units', 16, 64, step=16),
        activation='relu'
    ))
    
    model.add(Dense(1))  # Fixed output
    
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])
    )
    
    model.compile(optimizer=optimizer, loss=weighted_mse_large_moves)
    
    return model

# Setup BayesianOptimization tuner
tuner = BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=30,            # You can adjust: more trials = better search
    directory='tuner_results',
    project_name='bilstm_bayesian'
)

# EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.001,
    mode='min',
    restore_best_weights=True,
    verbose=1
)
epochs = 100
batch_size = 32

# Start search
tuner.search(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,   # fixed
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=2
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]


Trial 30 Complete [00h 31m 37s]
val_loss: 4.023556232452393

Best val_loss So Far: 3.733747959136963
Total elapsed time: 19h 21m 44s


/home/maked/CM3203-Project/CM3203-Venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
from helper_methods import save_model_with_config

save_model_with_config(
    model=best_model,
    tuner=tuner,
    features_config=features_config,
    time_bucket_folder=time_bucket_folder,
    test_size=test_size,
    early_stopping=early_stopping,
    X_train=X_train,
    y_train=y_train,
    epochs=epochs,
    batch_size=batch_size
)

Model saved to trained_models/lstm_16/model.keras
Configuration saved to trained_models/lstm_16/config.json

All model artifacts saved to trained_models/lstm_16
